In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
train_size  = 4096
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_test = x_train.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
layers = tf.keras.layers

In [9]:
gaussian_initializer=tf.keras.initializers.GlorotNormal()

In [10]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(img_input)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
out = layers.Dense(10, kernel_initializer=gaussian_initializer)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [11]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-4
    elif epoch < 55:
        return 1e-2
    elif epoch < 105:
        return 1e-2
    else:
        return 1e-4

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
              loss=tf.keras.losses.MSE,
              metrics=['accuracy'])

In [13]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback])

Epoch 1/200
15/15 [==============================] - 7s 459ms/step - loss: 0.1000 - accuracy: 0.1072 - val_loss: 0.1000 - val_accuracy: 0.1366
Epoch 2/200
15/15 [==============================] - 1s 66ms/step - loss: 0.1000 - accuracy: 0.1099 - val_loss: 0.1000 - val_accuracy: 0.1415
Epoch 3/200
15/15 [==============================] - 3s 203ms/step - loss: 0.1000 - accuracy: 0.1093 - val_loss: 0.0999 - val_accuracy: 0.1341
Epoch 4/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0999 - accuracy: 0.1069 - val_loss: 0.0999 - val_accuracy: 0.1415
Epoch 5/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0999 - accuracy: 0.1077 - val_loss: 0.0999 - val_accuracy: 0.1268
Epoch 6/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0993 - accuracy: 0.1017 - val_loss: 0.0986 - val_accuracy: 0.1463
Epoch 7/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0981 - accuracy: 0.1047 - val_loss: 0.0974 - val_accuracy: 0.1463
Epoc

15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 59/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 60/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 61/200
15/15 [==============================] - 1s 34ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 62/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 63/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 64/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 65/200

15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 116/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 117/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 118/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 119/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 120/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 121/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 

Epoch 172/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 173/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 174/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 175/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 176/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 177/200
15/15 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy: 0.1463
Epoch 178/200
15/15 [==============================] - 1s 34ms/step - loss: 0.0900 - accuracy: 0.1047 - val_loss: 0.0900 - val_accuracy:

In [15]:
model.save('./model_weights/vgg_19_train=4096_mse_init=gaussian.h5')